## ML pipeline deployment using TFX
To simulate a real world application of ML solution, I'm deploying a model using TFX

According to Google: *“A TFX pipeline is a sequence of components that implement an ML pipeline which is specifically designed for scalable, high-performance machine learning tasks. That includes modeling, training, serving inference, and managing deployments to online, native mobile, and JavaScript targets.”*

In [1]:
import pandas as pd

In [2]:
import tensorflow as tf
print(f'Tensorflow version: {tf.__version__}')

from tfx import v1 as tfx
print(f'TFX version: {tfx.__version__}')

Tensorflow version: 2.8.0
TFX version: 1.7.1


In [3]:
import os

PIPELINE_NAME = "penguin-simple"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [4]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

('C:\\Users\\jongm\\AppData\\Local\\Temp\\tfx-data43ty87jb\\data.csv',
 <http.client.HTTPMessage at 0x14fd879a070>)

### Data

In [5]:
pd.read_csv(_data_filepath).head()

,species,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
0,0,0.254545,0.666667,0.152542,0.291667
1,0,0.269091,0.511905,0.237288,0.305556
2,0,0.298182,0.583333,0.389831,0.152778
3,0,0.167273,0.738095,0.355932,0.208333
4,0,0.261818,0.892857,0.305085,0.263889


In [16]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

example_gen = tfx.components.CsvExampleGen(input_base=_data_filepath)
context.run(example_gen, enable_cache=True)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data C:\Users\jongm\AppData\Local\Temp\tfx-data43ty87jb\data.csv\* to TFExample.


RuntimeError: Split pattern C:\Users\jongm\AppData\Local\Temp\tfx-data43ty87jb\data.csv\* does not match any files.

In [12]:
from tfx.components import CsvExampleGen
from tfx.utils import dsl_utils
from tfx.proto import example_gen_pb2

output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1),
        example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=1)
    ])
)

examples = dsl_utils.external_input('data')
example_gen = CsvExampleGen(input=examples, output_config=output)

context.run(example_gen)

ImportError: cannot import name 'dsl_utils' from 'tfx.utils' (C:\ProgramData\Anaconda3\envs\main\lib\site-packages\tfx\utils\__init__.py)

In [10]:
output

split_config {
  splits {
    name: "train"
    hash_buckets: 8
  }
  splits {
    name: "eval"
    hash_buckets: 1
  }
  splits {
    name: "test"
    hash_buckets: 1
  }
}

### Data Ingestion

### Data Validation

### Feature Engineering

### Train Model

### Validate Model

### Push If Good

### Serve Model